In [1]:
from llama_index.legacy import SimpleDirectoryReader, StorageContext, ServiceContext
from llama_index.legacy.indices.vector_store import VectorStoreIndex
from llama_iris import IRISVectorStore

In [2]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
documents = SimpleDirectoryReader("../data/support").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: d265d192-84f5-4da9-bbe5-a1f911fbd06b


In [4]:
username = 'demo'
password = 'demo' 
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [34]:
vector_store = IRISVectorStore.from_params(
    connection_string=CONNECTION_STRING,
    table_name="paul_graham_essay",
    embed_dim=1536,  # openai embedding dimension
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# service_context = ServiceContext.from_defaults(
#     embed_model=embed_model, llm=None
# )

In [35]:
# This creates a persistent vector store (as a SQL table) under the hood and stores the document embeddings. This should only be called once!
index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
    show_progress=True, 
    # service_context=service_context,
)
query_engine = index.as_query_engine()

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


In [11]:
# If reconnecting to the vector store, use this: 

# index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# query_engine = index.as_query_engine()



In [18]:
# To add documents to an existing vector store:

# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# for doc in documents:
#     index.insert(document=doc, storage_context=storage_context)

In [40]:
response = query_engine.query("Wholegrain standards used in the recipe?")

In [41]:
import textwrap
print(textwrap.fill(str(response), 100))

Wholegrain standards used in the recipe include wholemeal bread (≥25% wholegrains), wholemeal pita
(≥25% wholegrains), wholemeal wraps (≥25% wholegrains), wholemeal buns and rolls (≥10% wholegrains),
rice/dehydrated rice (≥20% unpolished rice), glutinous rice (≥20% unpolished rice), rice flour (≥20%
unpolished rice), dry wheat noodles (≥15% whole wheat), dry rice vermicelli (≥80% brown rice), pasta
(whole durum wheat pasta/wholemeal pasta/semolina), wholemeal pizza base (25% wholegrains), and
fresh oriental rice/wheat noodles (≥15% wholegrains).


In [ ]:
response = query_engine.query("What happened in the mid 1980s?")
print(textwrap.fill(str(response), 100))

AI was in the air in the mid 1980s, and two things that influenced the individual were a novel by
Heinlein called The Moon is a Harsh Mistress, featuring an intelligent computer called Mike, and a
PBS documentary showing Terry Winograd using SHRDLU.
